# 第8章: ニューラルネット
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ

## 70. 単語ベクトルの和による特徴量

In [1]:
import gensim
text = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin", binary=True)

In [2]:
import numpy as np
import pandas as pd
train_article=pd.read_csv('train.csv')
val_article=pd.read_csv('val.csv')
test_article=pd.read_csv('test.csv')

In [3]:
def create_X_vec(article):
    article_vec_accumulator=np.zeros((len(article),300))
    counter=-1
    for article_list in article.TITLE.tolist():
        article_list=article_list.split(" ")
        counter+=1
        predicted_value=[]
        for word in article_list:
            word=word.replace("'","")
            try:
                predicted_value.append(text[word])
            except KeyError:
                pass
        article_vec_mean=np.array(predicted_value).mean(axis=0)
        article_vec_accumulator[counter,:]=article_vec_mean
    return article_vec_accumulator

In [4]:
category_label_dict={
    "b":0,
    "t":1,
    "e":2,
    "m":3
}

In [89]:
def create_y_sc(article):
    article_sc_accumulator=np.zeros((len(article),1),dtype=int)
    counter=-1
    for category in article.CATEGORY.tolist():
        label_id=category_label_dict[category]
        counter+=1
        article_sc_accumulator[counter,:]=label_id
    return article_sc_accumulator

In [90]:
X_train=create_X_vec(article=train_article)
X_val=create_X_vec(article=val_article)
X_test=create_X_vec(article=test_article)

In [91]:
y_train=create_y_sc(article=train_article)
y_val=create_y_sc(article=val_article)
y_test=create_y_sc(article=test_article)

## 71. 単層ニューラルネットワークによる予測

In [218]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [228]:
model = Sequential()
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [248]:
y_pred=model.predict(X_train[:4,:])

## 72. 損失と勾配の計算

In [249]:
y_true=y_train_one_hot[:4,:]

In [250]:
from keras import losses
loss=losses.categorical_crossentropy(y_true, y_pred)

In [251]:
loss.numpy()

array([1.2816622 , 0.01001023, 1.3824099 , 1.1875353 ], dtype=float32)

In [263]:
from keras import backend as K
variables=X_train[:4,:]
grad=K.gradients(loss, model.inputs)

TypeError: Tensor is unhashable if Tensor equality is enabled. Instead, use tensor.experimental_ref() as the key.

In [266]:
get_grad

[None]

## 73. 確率的勾配降下法による学習


In [234]:
model.fit(X_train, y_train_one_hot, epochs=100)

Epoch 1/100
10685/10685 [==============================] - 0s 35us/step - loss: 1.2066 - accuracy: 0.5928
Epoch 2/100
10685/10685 [==============================] - 0s 34us/step - loss: 1.0358 - accuracy: 0.7546
Epoch 3/100
10685/10685 [==============================] - 0s 34us/step - loss: 0.9547 - accuracy: 0.7670
Epoch 4/100
10685/10685 [==============================] - 0s 31us/step - loss: 0.8979 - accuracy: 0.7700
Epoch 5/100
10685/10685 [==============================] - 0s 33us/step - loss: 0.8540 - accuracy: 0.7719
Epoch 6/100
10685/10685 [==============================] - 0s 36us/step - loss: 0.8184 - accuracy: 0.7727
Epoch 7/100
10685/10685 [==============================] - 0s 33us/step - loss: 0.7887 - accuracy: 0.7740
Epoch 8/100
10685/10685 [==============================] - 0s 33us/step - loss: 0.7636 - accuracy: 0.7753
Epoch 9/100
10685/10685 [==============================] - 0s 36us/step - loss: 0.7417 - accuracy: 0.7759
Epoch 10/100
10685/10685 [====================

10685/10685 [==============================] - 0s 37us/step - loss: 0.4243 - accuracy: 0.8601
Epoch 78/100
10685/10685 [==============================] - 1s 51us/step - loss: 0.4230 - accuracy: 0.8614
Epoch 79/100
10685/10685 [==============================] - 1s 58us/step - loss: 0.4217 - accuracy: 0.8626
Epoch 80/100
10685/10685 [==============================] - 0s 46us/step - loss: 0.4204 - accuracy: 0.8633
Epoch 81/100
10685/10685 [==============================] - 1s 54us/step - loss: 0.4192 - accuracy: 0.8639
Epoch 82/100
10685/10685 [==============================] - 1s 53us/step - loss: 0.4180 - accuracy: 0.8644
Epoch 83/100
10685/10685 [==============================] - 0s 46us/step - loss: 0.4168 - accuracy: 0.8654
Epoch 84/100
10685/10685 [==============================] - 0s 38us/step - loss: 0.4156 - accuracy: 0.8656
Epoch 85/100
10685/10685 [==============================] - 0s 32us/step - loss: 0.4144 - accuracy: 0.8664
Epoch 86/100
10685/10685 [========================

## 74. 正解率の計測

In [241]:
from sklearn.metrics import accuracy_score
print(f"train acc: {accuracy_score(y_train,model.predict_classes(X_train))}")
print(f"test acc: {accuracy_score(y_test,model.predict_classes(X_test))}")

train acc: 0.871408516612073
test acc: 0.8764044943820225
